In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Wed Jul 27 10:07:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import shutil
import sys

In [ ]:
#Hos pair BERT polarity
# train = '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseB/Res/Res_train_pair_phaseB.csv'
# dev = '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseB/Res/Res_dev_pair_phaseB.csv'
# test = '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseB/Res/Res_test_pair_phaseB.csv'

In [ ]:
#Hos pair Bert polarrity
train = '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseB/Hos/Hos_train_pair_PhaseB.csv'
dev = '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseB/Hos/Hos_dev_pair_PhaseB.csv'
test = '/content/drive/MyDrive/Đồ án tốt nghiệp/Data/PhaseB/Hos/Hos_test_pair_PhaseB.csv'

In [ ]:
train = pd.read_csv(train)
dev = pd.read_csv(dev)
test = pd.read_csv(test)

In [ ]:
print(len(train.columns))
print(len(dev.columns))
print(len(test.columns))

6
6
6


In [ ]:
def processAspect(x):
  x = x.replace('#', ' ')
  x = x.replace('&', ' ')
  return ''.join(x)

In [ ]:
train

,id,content,Aspect,positive,negative,neutral
0,﻿#1,Rộng rãi khách sạn mới nhưng rất vắng. Các dịc...,HOTEL#DESIGN&FEATURES,1,0,0
1,﻿#1,Rộng rãi khách sạn mới nhưng rất vắng. Các dịc...,HOTEL#GENERAL,0,1,0
2,#2,Địa điểm thuận tiện trong vòng bán kính 1 giá ...,LOCATION#GENERAL,1,0,0
3,#3,Phục vụ view đẹp vị trí,SERVICE#GENERAL,1,0,0
4,#3,Phục vụ view đẹp vị trí,HOTEL#GENERAL,1,0,0
...,...,...,...,...,...,...
13944,#2999,Đầu tiên về ưu điểm Phòng sạch sẽ nội thất mới...,HOTEL#GENERAL,0,1,0
13945,#3000,Tôi ở đây 1 đêm và thấy khách sạn khá đẹp lạ v...,HOTEL#DESIGN&FEATURES,1,0,0
13946,#3000,Tôi ở đây 1 đêm và thấy khách sạn khá đẹp lạ v...,HOTEL#COMFORT,1,0,0
13947,#3000,Tôi ở đây 1 đêm và thấy khách sạn khá đẹp lạ v...,SERVICE#GENERAL,1,0,0


In [ ]:
train.columns

Index(['id', 'content', 'Aspect', 'positive', 'negative', 'neutral'], dtype='object')

In [ ]:
target_list = ['positive', 'negative', 'neutral']

In [ ]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 1e-05
NUM_LABELS = len(target_list)

In [ ]:
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer, AutoModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

In [ ]:
# text_a = "Quán ăn này rất ngon, sẽ quay lại ủng hộ"
# text_b = 'FOOD STYLE OPTIONS'
# encoding = tokenizer.encode_plus(
#     text_a,
#     text_b,
#     add_special_tokens = True,
#     max_length=MAX_LEN,
#     padding = 'max_length',
#     truncation='only_first',
#     return_tensors='pt'
# )
# encoding

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, df, tokenizer, max_len):
    self.df = df
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.text_a = self.df['content']
    self.text_b = self.df['Aspect']
    self.targets = self.df[target_list].values

  def __len__(self):
    return len(self.text_a)
    
  def __getitem__(self, index):
    text_a = str(self.text_a[index])
    text_a = " ".join(text_a.split())
    text_b = str(self.text_b[index])
    text_b = processAspect(text_b)
    text_b = " ".join(text_b.split())

    inputs = self.tokenizer.encode_plus(
        text_a,
        text_b,
        add_special_tokens = True,
        max_length = self.max_len,
        padding = 'max_length',
        return_token_type_ids = True,
        truncation='longest_first',
        return_attention_mask = True,
        return_tensors = 'pt'
    )
    return {
        'input_ids':inputs['input_ids'].flatten(),
        'attention_mask': inputs['attention_mask'].flatten(),
        'token_type_ids': inputs['token_type_ids'].flatten(),
        'targets': torch.FloatTensor(self.targets[index])
    }

In [ ]:
train_dataset = CustomDataset(train, tokenizer, MAX_LEN)
val_dataset = CustomDataset(dev, tokenizer, MAX_LEN)

In [ ]:
train_dataset.targets

array([[1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [ ]:
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle = True,
    batch_size = TRAIN_BATCH_SIZE,
    num_workers = 0
)
val_data_loader = torch.utils.data.DataLoader(
    val_dataset,
    shuffle = False,
    batch_size = VALID_BATCH_SIZE,
    num_workers = 0
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
  checkpoint = torch.load(checkpoint_fpath)
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  valid_loss_min = checkpoint['valid_loss_min']
  return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
  f_path = checkpoint_path
  torch.save(state, f_path)
  if is_best:
    best_fpath = best_model_path
    shutil.copyfile(f_path, best_fpath)

In [ ]:
class BERTClass(nn.Module):
  def __init__(self):
    super(BERTClass, self).__init__()
    self.bert_model = BertModel.from_pretrained('bert-base-multilingual-cased', return_dict=True)
    # self.bert_model = AutoModel.from_pretrained('vinai/phobert-base', return_dict=True)

    self.dropout = nn.Dropout(0.1)
    self.linear = nn.Linear(768, NUM_LABELS) 
  def forward(self, input_ids, attention_mask, token_type_ids):
    output = self.bert_model(input_ids, attention_mask, token_type_ids)
    output_dropout = self.dropout(output.pooler_output)
    output = self.linear(output_dropout)
    return output

In [ ]:
model = BERTClass()
model.to(device)

In [ ]:
def loss_fn(outputs, targets):
  return nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params = model.parameters(), lr = LEARNING_RATE)

In [ ]:
def train_model(
    n_epochs,
    training_loader,
    validation_loader,
    model,
    optimizer,
    checkpoint_path,
    best_model_path
):
  valid_loss_min = np.Inf
  for epoch in range(1, n_epochs + 1):
    print('Epoch {}/{}'.format(epoch, n_epochs))
    train_loss = 0
    valid_loss = 0
    model.train()
    #training loop
    for index, batch in enumerate(train_data_loader):
      input_ids = batch['input_ids'].to(device, dtype = torch.long)
      attention_mask = batch['attention_mask'].to(device, dtype = torch.long)
      token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
      targets = batch['targets'].to(device, dtype = torch.float)
      outputs = model(input_ids, attention_mask, token_type_ids)
      optimizer.zero_grad()
      loss = loss_fn( outputs, targets)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      train_loss = train_loss + (1/(index+1)*(loss.item()-train_loss))
    #validation loop
    model.eval()
    with torch.no_grad():
      for index, batch in enumerate(train_data_loader):
        input_ids = batch['input_ids'].to(device, dtype = torch.long)
        attention_mask = batch['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.float)
        outputs = model(input_ids, attention_mask, token_type_ids)
        loss = loss_fn(outputs, targets)
        valid_loss = valid_loss + (1/(index+1)*(loss.item()-train_loss))
    checkpoint = {
        'epoch': epoch+1,
        'valid_loss_min': valid_loss,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    save_ckp(checkpoint, False, checkpoint_path, best_model_path)
    print('Loss: {}, Val_Loss:{}'.format(train_loss, valid_loss))
  return model

In [ ]:
import time

In [ ]:
start_time = time.time()
trained_model = train_model(EPOCHS, train_data_loader,  val_data_loader, model, optimizer, "/curr_ckpt", "/best.pt" )
elapsed_time_secs = time.time() - start_time
elapsed_time_secs

In [ ]:
torch.save(trained_model.state_dict(), '/content/drive/MyDrive/Đồ án tốt nghiệp/Save Pre-Train Model/HosCombine.pt')

In [ ]:
# testing
def get_predict(review, aspect):
  text_a = review
  text_b = processAspect(aspect)
  encodings = tokenizer.encode_plus(
      text_a,
      text_b,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  trained_model.eval()

  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output = trained_model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      t = np.array(final_output)
      # THRESHOLD = 0.4
      # upper, lower = 1, 0

      # final_out = np.where(t > THRESHOLD, upper, lower)
  return t

Evaluate

In [ ]:
dev_predicts = []
for i in range(len(dev['content'].values)):
  dev_predicts.append(get_predict(dev['content'][i], dev['Aspect'][i])[0])

In [ ]:
test_predicts = []
for i in range(len(test['content'].values)):
  test_predicts.append(get_predict(test['content'][i], test['Aspect'][i])[0])

In [ ]:
THRESHOLD = 0.4
upper, lower = 1, 0

In [ ]:
dev_predicts = [np.where(dev_predicts[i] > THRESHOLD, upper, lower).tolist() for i in range(len(dev_predicts))]
test_predicts = [np.where(test_predicts[i] > THRESHOLD, upper, lower).tolist() for i in range(len(test_predicts))]

In [ ]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix

In [ ]:
print(classification_report(dev[target_list].values, dev_predicts, target_names = target_list))

In [ ]:
print(classification_report(test[target_list].values, test_predicts, target_names = target_list))

              precision    recall  f1-score   support

    positive       0.93      0.93      0.93      1806
    negative       0.77      0.89      0.82       645
     neutral       0.46      0.31      0.37       133

   micro avg       0.87      0.88      0.88      2584
   macro avg       0.72      0.71      0.71      2584
weighted avg       0.87      0.88      0.87      2584
 samples avg       0.88      0.88      0.88      2584



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
